In [1]:
!pip install nibabel
!pip install -U --user nilearn
!python -m pip install pyradiomics
!pip install SimpleITK   

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 kB 896.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 3.9 MB/s eta 0:00:00


In [2]:
import os
import shutil
import tarfile
from itertools import chain
from collections import defaultdict
from tqdm import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import plotly
import plotly.graph_objects as go

import nibabel as nib
from nilearn import image, plotting
import SimpleITK as sitk
from SimpleITK import GetImageFromArray
import radiomics
from radiomics.featureextractor import RadiomicsFeatureExtractor

import warnings
warnings.filterwarnings('ignore')

import albumentations as A

from sklearn.preprocessing import OneHotEncoder, LabelEncoder,MinMaxScaler,StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score

from catboost import Pool, CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
import xgboost as xgb

import logging
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)

import pickle
import json

## Extracting Data

In [3]:
train_df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
data_path = "/kaggle/working/data"

# Create Data Directory
if not os.path.isdir("/kaggle/working/data"):
    os.makedirs("/kaggle/working/data", exist_ok=True)

# # Extract Update
# print("\n... Extracting BraTSID=00495 Task1 Update Files ...")
# tar = tarfile.open("/kaggle/input/brats-2021-task1/BraTS2021_00495.tar")
# tar.extractall("/kaggle/working/data")
# tar.close()

# # Extract Update
# print("... Extracting BraTSID=00621 Task1 Update Files ...")
# tar = tarfile.open("/kaggle/input/brats-2021-task1/BraTS2021_00621.tar")
# tar.extractall("/kaggle/working/data")
# tar.close()

# Extract Main Training Data
print("... Extracting Main Task1 Training Files (3-5 Minutes) ...\n")
tar = tarfile.open("/kaggle/input/brats-2021-task1/BraTS2021_Training_Data.tar")
tar.extractall("/kaggle/working/data")
tar.close()

os.remove('./data/.DS_Store')

... Extracting Main Task1 Training Files (3-5 Minutes) ...



## Inititalization

In [4]:
rm_ls = [1024, 1026, 611, 1028, 1034, 12, 1552, 1041, 18, 19, 1044, 532, 1558, 17, 1560, 1048, 1562, 1561, 538, 1053, 1565, 1567, 31, 1569, 28, 540, 1572, 1573, 1062, 1061, 551, 1577, 1578, 1066, 1579, 1065, 1070, 45, 1584, 561, 49, 1588, 1076, 52, 1591, 567, 1593, 1594, 570, 1595, 1084, 1597, 1087, 576, 1088, 578, 1600, 580, 575, 582, 583, 1605, 1089, 74, 68, 1101, 1102, 1104, 1617, 1618, 1619, 1106, 1616, 1110, 1111, 1624, 89, 602, 85, 1116, 604, 601, 1631, 607, 1633, 1122, 1635, 1636, 1637, 101, 1639, 612, 1641, 99, 613, 618, 621, 622, 1647, 620, 1138, 1651, 1652, 1653, 118, 1654, 1656, 630, 1658, 628, 1660, 631, 1152, 640, 1666, 1154, 1155, 645, 646, 1153, 649, 1162, 652, 654, 655, 657, 659, 661, 1177, 1178, 1179, 1180, 668, 1182, 1186, 1188, 1189, 677, 679, 1192, 1195, 171, 1198, 693, 694, 185, 186, 703, 192, 705, 1219, 707, 709, 199, 1225, 715, 206, 207, 212, 214, 728, 218, 734, 737, 228, 740, 742, 242, 757, 759, 250, 765, 1279, 772, 1287, 778, 53, 269, 270, 271, 791, 281, 793, 1308, 797, 286, 1311, 1313, 1314, 803, 804, 805, 1319, 59, 1322, 811, 1324, 1326, 304, 305, 1330, 61, 1339, 63, 1342, 834, 325, 838, 1351, 1360, 1361, 1362, 339, 1085, 1365, 1366, 344, 1369, 1370, 1368, 350, 351, 1376, 1377, 352, 1380, 1381, 1383, 1388, 1391, 1400, 378, 382, 1413, 1414, 1415, 1416, 1418, 1419, 1421, 1422, 1424, 1426, 1427, 1428, 1429, 1430, 1431, 1432, 1434, 1437, 1438, 418, 432, 444, 1469, 456, 528, 488, 1001, 493, 495, 498, 1013, 1015, 504, 1017, 1021, 511]

In [5]:
################
## Training Data
################

train_path =  "./data" #"../input/brats-2021-training-data-extracted/data" #"./data"
train_names = os.listdir(train_path)

id = -1

train_t1 = [os.path.join(os.path.join(train_path,f), f+"_t1.nii.gz") for f in os.listdir(train_path)]#[:id]
train_t1ce = [os.path.join(os.path.join(train_path,f), f+"_t1ce.nii.gz") for f in os.listdir(train_path)]#[:id]
train_t2 = [os.path.join(os.path.join(train_path,f), f+"_t2.nii.gz") for f in os.listdir(train_path)]#[:id]
train_flair = [os.path.join(os.path.join(train_path,f), f+"_flair.nii.gz") for f in os.listdir(train_path)]#[:id]
train_seg = [os.path.join(os.path.join(train_path,f), f+"_seg.nii.gz") for f in os.listdir(train_path)]#[:id]

id_ls = [int(i.split("/")[-2].split("_")[-1]) for i in train_t1]

# train_t1 = [os.path.join(i,os.listdir(i)[0]) for c,i in enumerate(train_t1) if id_ls[c] not in rm_ls]
# train_t1ce = [os.path.join(i,os.listdir(i)[0]) for c,i in enumerate(train_t1ce) if id_ls[c] not in rm_ls]
# train_t2 = [os.path.join(i,os.listdir(i)[0]) for c,i in enumerate(train_t2) if id_ls[c] not in rm_ls]
# train_flair = [os.path.join(i,os.listdir(i)[0]) for c,i in enumerate(train_flair) if id_ls[c] not in rm_ls]
# train_seg = [os.path.join(i,os.listdir(i)[0]) for c,i in enumerate(train_seg) if id_ls[c] not in rm_ls]

# new_id_ls = [i for i in id_ls if i not in rm_ls]
train_df = pd.DataFrame.from_dict({
    "id" : id_ls,
    "BraTS21ID" : id_ls,
    "t1" : train_t1,
    "t1ce" : train_t1ce,
    "t2" : train_t2,
    "flair" : train_flair,
    "seg" : train_seg
})
train_df.set_index("id", inplace=True)
train_df.head()

,BraTS21ID,t1,t1ce,t2,flair,seg
id,,,,,,
658,658,./data/BraTS2021_00658/BraTS2021_00658_t1.nii.gz,./data/BraTS2021_00658/BraTS2021_00658_t1ce.ni...,./data/BraTS2021_00658/BraTS2021_00658_t2.nii.gz,./data/BraTS2021_00658/BraTS2021_00658_flair.n...,./data/BraTS2021_00658/BraTS2021_00658_seg.nii.gz
620,620,./data/BraTS2021_00620/BraTS2021_00620_t1.nii.gz,./data/BraTS2021_00620/BraTS2021_00620_t1ce.ni...,./data/BraTS2021_00620/BraTS2021_00620_t2.nii.gz,./data/BraTS2021_00620/BraTS2021_00620_flair.n...,./data/BraTS2021_00620/BraTS2021_00620_seg.nii.gz
1293,1293,./data/BraTS2021_01293/BraTS2021_01293_t1.nii.gz,./data/BraTS2021_01293/BraTS2021_01293_t1ce.ni...,./data/BraTS2021_01293/BraTS2021_01293_t2.nii.gz,./data/BraTS2021_01293/BraTS2021_01293_flair.n...,./data/BraTS2021_01293/BraTS2021_01293_seg.nii.gz
583,583,./data/BraTS2021_00583/BraTS2021_00583_t1.nii.gz,./data/BraTS2021_00583/BraTS2021_00583_t1ce.ni...,./data/BraTS2021_00583/BraTS2021_00583_t2.nii.gz,./data/BraTS2021_00583/BraTS2021_00583_flair.n...,./data/BraTS2021_00583/BraTS2021_00583_seg.nii.gz
1414,1414,./data/BraTS2021_01414/BraTS2021_01414_t1.nii.gz,./data/BraTS2021_01414/BraTS2021_01414_t1ce.ni...,./data/BraTS2021_01414/BraTS2021_01414_t2.nii.gz,./data/BraTS2021_01414/BraTS2021_01414_flair.n...,./data/BraTS2021_01414/BraTS2021_01414_seg.nii.gz


In [6]:
# ##################
# ## Validation Data
# ##################

# val_path = "../input/brats-2021-validation-data"
# label_path = "../input/brats-2021-nnunet-training/setup/results/final_preds"

# id_ls = [i.split(".")[0] for i in os.listdir(label_path)]

# val_t1 = [f"{val_path}/{f}/{f}_t1.nii" for f in id_ls]
# val_t1ce = [f"{val_path}/{f}/{f}_t1ce.nii" for f in id_ls]
# val_t2 = [f"{val_path}/{f}/{f}_t2.nii" for f in id_ls]
# val_flair = [f"{val_path}/{f}/{f}_flair.nii" for f in id_ls]
# val_seg = [f"{label_path}/{f}.nii.gz" for f in id_ls]

# val_t1 = [os.path.join(i,os.listdir(i)[0]) for c,i in enumerate(val_t1)]
# val_t1ce = [os.path.join(i,os.listdir(i)[0]) for c,i in enumerate(val_t1ce)]
# val_t2 = [os.path.join(i,os.listdir(i)[0]) for c,i in enumerate(val_t2)]
# val_flair = [os.path.join(i,os.listdir(i)[0]) for c,i in enumerate(val_flair)]

# train_df = pd.DataFrame.from_dict({
#     "id" : id_ls,
#     "BraTS21ID" : id_ls,
#     "t1" : val_t1,
#     "t1ce" : val_t1ce,
#     "t2" : val_t2,
#     "flair" : val_flair,
#     "seg" : val_seg
# })
# train_df.set_index("id", inplace=True)
# train_df.head()

In [7]:
# for c,i in enumerate(train_seg):
#     try:
#         x = os.path.join(i,os.listdir(i)[0])
#     except:
#         rm_ls.append(id_ls[c])
        
# print(len(rm_ls))

## Image Reader

In [8]:
class ImageReader:
    def __init__(self, img_size=256, normalize=False, single_class=False, class_ = "multi"):
        pad_size = 256 if img_size > 256 else 224
        self.resize = A.Compose(
            [
                A.PadIfNeeded(min_height=pad_size, min_width=pad_size, value=0),
                A.Resize(img_size, img_size)
            ]
        )
        self.normalize=normalize
        self.single_class=single_class
        self.class_ = class_
        
    def read_file(self, path) :
        scan_type = path.split('_')[-1]
        #scan_type = path.split("/")[-2].split("_")[-1]
        raw_image = nib.load(path).get_fdata()
        mask_path = path.replace(scan_type, 'seg.nii.gz')
        #id_ = "_".join(path.split("/")[-2].split("_")[:2])
        #mask_path = '../input/brats-2021-nnunet-training/setup/results/final_preds/' + id_ + '.nii.gz'
        raw_mask = nib.load(mask_path).get_fdata()
        processed_frames, processed_masks = [], []
        for frame_idx in range(raw_image.shape[2]):
            frame = raw_image[:, :, frame_idx]
            mask = raw_mask[:, :, frame_idx]
            if self.normalize:
                if frame.max() > 0:
                    frame = frame/frame.max()
                frame = frame.astype(np.float32)
            else:
                frame = frame.astype(np.uint8)
            resized = self.resize(image=frame, mask=mask)
            processed_frames.append(resized['image'])
            #processed_masks.append(1*(resized['mask'] > 0) if self.single_class else resized['mask'])
            if self.class_ == "multi":
                processed_masks.append(1*(resized['mask'] > 0))
            elif self.class_ == 1:
                processed_masks.append(1*(resized['mask'] == 1))
            elif self.class_ == 2:
                processed_masks.append(1*(resized['mask'] == 2))
            else:
                processed_masks.append(1*(resized['mask'] == 4))
        return {
            'scan': np.stack(processed_frames, 0),
            'segmentation': np.stack(processed_masks, 0),
            'orig_shape': raw_image.shape
        }
    
    def load_patient_scan(self, idx, modality = 'flair') :
        return self.read_file(train_df.loc[idx][modality])

## Tumor viewer

In [9]:
def generate_3d_scatter(
    x:np.array, y:np.array, z:np.array, colors:np.array,
    size:int=3, opacity:float=0.2, scale:str='Teal',
    hover:str='skip', name:str='MRI'
) -> go.Scatter3d:
    return go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers', hoverinfo=hover,
        marker = dict(
            size=size, opacity=opacity,
            color=colors, colorscale=scale
        ),
        name=name
    )


class ImageViewer3d():
    def __init__(
        self, reader:ImageReader, mri_downsample:int=10, mri_colorscale:str='Ice'
    ) -> None:
        self.reader = reader
        self.mri_downsample = mri_downsample
        self.mri_colorscale = mri_colorscale

    def load_clean_mri(self, image:np.array, orig_dim:int) -> dict:
        shape_offset = image.shape[1]/orig_dim
        z, x, y = (image > 0).nonzero()
        # only (1/mri_downsample) is sampled for the resulting image
        x, y, z = x[::self.mri_downsample], y[::self.mri_downsample], z[::self.mri_downsample]
        colors = image[z, x, y]
        return dict(x=x/shape_offset, y=y/shape_offset, z=z, colors=colors)
    
    def load_tumor_segmentation(self, image:np.array, orig_dim:int) -> dict:
        tumors = {}
        shape_offset = image.shape[1]/orig_dim
        # 1/1, 1/3 and 1/5 pixels for tumor tissue classes 1(core), 2(invaded) and 4(enhancing)
        sampling = {
            1: 1, 2: 3, 4: 5
        }
        for class_idx in sampling:
            z, x, y = (image == class_idx).nonzero()
            x, y, z = x[::sampling[class_idx]], y[::sampling[class_idx]], z[::sampling[class_idx]]
            tumors[class_idx] = dict(
                x=x/shape_offset, y=y/shape_offset, z=z,
                colors=class_idx/4
            )
        return tumors
    
    def collect_patient_data(self, scan:dict) -> tuple:
        clean_mri = self.load_clean_mri(scan['scan'], scan['orig_shape'][0])
        tumors = self.load_tumor_segmentation(scan['segmentation'], scan['orig_shape'][0])
        markers_created = clean_mri['x'].shape[0] + sum(tumors[class_idx]['x'].shape[0] for class_idx in tumors)
        return [
            generate_3d_scatter(**clean_mri, scale=self.mri_colorscale, opacity=0.3, hover='skip', name='Brain MRI'),
            generate_3d_scatter(**tumors[1], opacity=0.8, hover='all', name='Necrotic tumor core'),
            generate_3d_scatter(**tumors[2], opacity=0.4, hover='all', name='Peritumoral invaded tissue'),
            generate_3d_scatter(**tumors[4], opacity=0.4, hover='all', name='GD-enhancing tumor'),
        ], markers_created
    
    def get_3d_scan(self, patient_idx:int, scan_type:str='flair') -> go.Figure:
        scan = self.reader.load_patient_scan(patient_idx, scan_type)
        data, num_markers = self.collect_patient_data(scan)
        fig = go.Figure(data=data)
        fig.update_layout(
            title=f"[Patient id:{patient_idx}] brain MRI scan ({num_markers} points)",
            legend_title="Pixel class (click to enable/disable)",
            font=dict(
                family="Courier New, monospace",
                size=14,
            ),
            margin=dict(
                l=0,r=0,b=0,t=30
            ),
            legend=dict(itemsizing='constant')
        )
        return fig

## Feature Extraction

In [10]:
reader = ImageReader(img_size=128, normalize=True, single_class=False, class_=1)
viewer = ImageViewer3d(reader, mri_downsample=25)

# fig = viewer.get_3d_scan(1228, 't1')
# plotly.offline.iplot(fig)

In [11]:
feature_names = [
    "BraTS21ID",\
    "diagnostics_Mask-original_BoundingBox", \
    "diagnostics_Mask-original_CenterOfMassIndex",\
    "diagnostics_Image-original_Mean", \
    "original_shape_Elongation",\
    "original_shape_Flatness", \
    "original_shape_LeastAxisLength",\
    "original_shape_MajorAxisLength",\
    "original_shape_Maximum2DDiameterColumn",\
    "original_shape_Maximum2DDiameterRow",\
    "original_shape_Maximum2DDiameterSlice",\
    "original_shape_Maximum3DDiameter",\
    "original_shape_MeshVolume",\
    "original_shape_MinorAxisLength",\
    "original_shape_Sphericity",\
    "original_shape_SurfaceArea",\
    "original_shape_SurfaceVolumeRatio",\
    "original_firstorder_10Percentile",\
    "original_firstorder_90Percentile",\
    "original_firstorder_Energy",\
#     "original_firstorder_Entropy",\
    "original_firstorder_InterquartileRange",\
    "original_firstorder_Kurtosis",\
    "original_firstorder_Maximum",\
    "original_firstorder_MeanAbsoluteDeviation",\
    "original_firstorder_Mean",\
    "original_firstorder_Median",\
    "original_firstorder_Minimum",\
    "original_firstorder_Range",\
    "original_firstorder_RobustMeanAbsoluteDeviation",\
    "original_firstorder_RootMeanSquared",\
    "original_firstorder_Skewness",\
    "original_firstorder_TotalEnergy",\
#     "original_firstorder_Uniformity",\
    "original_firstorder_Variance",\
#     "original_glcm_Autocorrelation",\
#     "original_glcm_ClusterProminence",\
#     "original_glcm_ClusterShade",\
#     "original_glcm_ClusterTendency",\
#     "original_glcm_Contrast",\
#     "original_glcm_Correlation",\
#     "original_glcm_DifferenceAverage",\
#     "original_glcm_DifferenceEntropy",\
#     "original_glcm_DifferenceVariance",\
#     "original_glcm_Id",\
#     "original_glcm_Idm",\
#     "original_glcm_Idmn",\
#     "original_glcm_Idn",\
#     "original_glcm_Imc1",\
#     "original_glcm_Imc2",\
#     "original_glcm_InverseVariance",\
#     "original_glcm_JointAverage",\
#     "original_glcm_JointEnergy",\
#     "original_glcm_JointEntropy",\
#     "original_glcm_MCC",\
#     "original_glcm_MaximumProbability",\
#     "original_glcm_SumAverage",\
#     "original_glcm_SumEntropy",\
#     "original_glcm_SumSquares",\
    "original_gldm_DependenceEntropy",\
    "original_gldm_DependenceNonUniformity",\
    "original_gldm_DependenceNonUniformityNormalized",\
    "original_gldm_DependenceVariance",\
    "original_gldm_GrayLevelNonUniformity",\
#     "original_gldm_GrayLevelVariance",\
#     "original_gldm_HighGrayLevelEmphasis",\
    "original_gldm_LargeDependenceEmphasis",\
    "original_gldm_LargeDependenceHighGrayLevelEmphasis",\
    "original_gldm_LargeDependenceLowGrayLevelEmphasis",\
#     "original_gldm_LowGrayLevelEmphasis",\
    "original_gldm_SmallDependenceEmphasis",\
    "original_gldm_SmallDependenceHighGrayLevelEmphasis",\
    "original_gldm_SmallDependenceLowGrayLevelEmphasis",\
    "original_glrlm_GrayLevelNonUniformity",\
#     "original_glrlm_GrayLevelNonUniformityNormalized",\
#     "original_glrlm_GrayLevelVariance",\
#     "original_glrlm_HighGrayLevelRunEmphasis",\
    "original_glrlm_LongRunEmphasis",\
    "original_glrlm_LongRunHighGrayLevelEmphasis",\
    "original_glrlm_LongRunLowGrayLevelEmphasis",\
#     "original_glrlm_LowGrayLevelRunEmphasis",\
    "original_glrlm_RunEntropy",\
    "original_glrlm_RunLengthNonUniformity",\
    "original_glrlm_RunLengthNonUniformityNormalized",\
    "original_glrlm_RunPercentage",\
    "original_glrlm_RunVariance",\
    "original_glrlm_ShortRunEmphasis",\
    "original_glrlm_ShortRunHighGrayLevelEmphasis",\
    "original_glrlm_ShortRunLowGrayLevelEmphasis",\
    "original_glszm_GrayLevelNonUniformity",\
#     "original_glszm_GrayLevelNonUniformityNormalized",\
#     "original_glszm_GrayLevelVariance",\
    "original_glszm_HighGrayLevelZoneEmphasis",\
    "original_glszm_LargeAreaEmphasis",\
#     "original_glszm_LargeAreaHighGrayLevelEmphasis",\
    "original_glszm_LargeAreaLowGrayLevelEmphasis",\
#     "original_glszm_LowGrayLevelZoneEmphasis",\
    "original_glszm_SizeZoneNonUniformity",\
    "original_glszm_SizeZoneNonUniformityNormalized",\
    "original_glszm_SmallAreaEmphasis",\
    "original_glszm_SmallAreaHighGrayLevelEmphasis",\
    "original_glszm_SmallAreaLowGrayLevelEmphasis",\
    "original_glszm_ZoneEntropy",\
    "original_glszm_ZonePercentage",\
    "original_glszm_ZoneVariance",\
#     "original_ngtdm_Busyness",\
#     "original_ngtdm_Coarseness",\
#     "original_ngtdm_Complexity",\
#     "original_ngtdm_Contrast",\
#     "original_ngtdm_Strength"
]

def get_radiomice_features(idx, reader, modality="flair"):
    data = reader.load_patient_scan(idx, modality)
    scan = sitk.GetImageFromArray(data["scan"])
    mask = sitk.GetImageFromArray(data["segmentation"])
    features = texture_extractor.execute(scan,mask)
    tmp_df = pd.DataFrame([features]).T
    row = [idx]
    col_names = ["BraTS21ID"]
    
    for col in feature_names[1:]:
        if "BoundingBox" in col:
            row.extend(list(tmp_df.loc[col][0]))
            for j in range(6):
                col_names.append(col+"_"+str(j))
        elif "CenterOfMassIndex" in col:
            row.extend(list(tmp_df.loc[col][0]))
            for j in range(3):
                col_names.append(col+"_"+str(j))
        else:
            row.append(tmp_df.loc[col][0])
            col_names.append(col)
        
    return row, col_names

In [12]:
rad_features = []
texture_extractor = RadiomicsFeatureExtractor(verbose=False)
for i in tqdm(id_ls[:10]):
    try:
        _, column_names = get_radiomice_features(i,reader)
        #rad_features.append(features)
    except Exception as e:
         print(i, e)

100%|██████████| 10/10 [00:09<00:00,  1.03it/s]


In [13]:
#####################
## Feature Extraction
#####################

texture_extractor = RadiomicsFeatureExtractor(verbose=False)
features = defaultdict(lambda : defaultdict(dict))

for modality in ["t1", "t2", "t1ce", "flair"]:
    for class_ in ["multi", 1, 2, 4]:
        reader = ImageReader(img_size=128, normalize=True, single_class=False, class_=class_)
        for i in tqdm(id_ls[:2]):
            try:
                tmp_features, column_names = get_radiomice_features(i,reader,modality) 
                features[modality][class_][i] = tmp_features
            except Exception as e:
                print(e)

100%|██████████| 2/2 [00:01<00:00,  1.33it/s]


In [14]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open('./training_radiomics_features.json', 'w') as f:
    json.dump(features, f, cls=NumpyEncoder)

In [15]:
with open('column_names.pkl', 'wb') as f:
    pickle.dump(column_names, f)

In [16]:
shutil.rmtree("./data")